In [24]:
import pandas as pd
from pathlib import Path

school_data_file = Path(r"C:\Users\Kyle_McDaniel_Python\Desktop\Columbia_Analytics_Bootcamp\pandas_challenge\Resources\schools_complete.csv")
student_data_file = Path(r"C:\Users\Kyle_McDaniel_Python\Desktop\Columbia_Analytics_Bootcamp\pandas_challenge\Resources\students_complete.csv")

In [25]:
school_data = pd.read_csv(school_data_file)
student_data = pd.read_csv(student_data_file)

In [26]:
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [27]:
school_count = school_data_complete['school_name'].nunique()
print(school_data_complete['school_name'].nunique())

15


In [28]:
student_count = school_data_complete['Student ID'].nunique()
print(school_data_complete['Student ID'].nunique())

39170


In [29]:
budget_total = school_data_complete['budget'].unique().sum()
print(budget_total)

24649428


In [30]:
math_avg = school_data_complete['math_score'].mean()
print(math_avg)

78.98537145774827


In [31]:
read_avg = school_data_complete['reading_score'].mean()
print(read_avg)

81.87784018381414


In [32]:
math_pass_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
math_pass_percent = math_pass_count / float(student_count) * 100
print(math_pass_percent)

74.9808526933878


In [33]:
read_pass_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
read_pass_percent = read_pass_count / float(student_count) * 100
print(read_pass_percent)

85.80546336482001


In [34]:
math_read_pass_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_pass_rate = math_read_pass_count /  float(student_count) * 100
overall_pass_rate

65.17232575950983

In [35]:
district_summary = {
    'Total Schools': [school_count],
    'Total Students': [student_count],
    'Total Budget': [budget_total],
    'Average Math Score': [math_avg],
    'Average Reading Score': [read_avg],
    '% Passing Math': [math_pass_percent],
    '% Passing Reading': [read_pass_percent],
    '% Overall Passing': [overall_pass_rate]
}

district_summary_df = pd.DataFrame(district_summary)

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [36]:
school_type = school_data.set_index(["school_name"])["type"]

In [37]:
merged_df = school_data_complete.merge(school_type, left_on='school_name', right_index=True)

In [38]:
per_school_students = merged_df.groupby('school_name')['Student ID'].nunique()

In [60]:
#per_school_budget = merged_df.groupby('school_name')['budget'].unique()

per_school_budget = pd.Series(merged_df.groupby('school_name')['budget'].unique())

print(per_school_budget)

per_school_capita = per_school_budget / per_school_students

school_name
Bailey High School       [3124928]
Cabrera High School      [1081356]
Figueroa High School     [1884411]
Ford High School         [1763916]
Griffin High School       [917500]
Hernandez High School    [3022020]
Holden High School        [248087]
Huang High School        [1910635]
Johnson High School      [3094650]
Pena High School          [585858]
Rodriguez High School    [2547363]
Shelton High School      [1056600]
Thomas High School       [1043130]
Wilson High School       [1319574]
Wright High School       [1049400]
Name: budget, dtype: object


In [40]:
per_school_math_avg = merged_df.groupby('school_name')['math_score'].mean()
per_school_reading_avg = merged_df.groupby('school_name')['reading_score'].mean()

In [41]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]["student_name"].count()
school_students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name").count()["student_name"]

In [42]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]["student_name"].count()
school_students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name").count()["student_name"]

In [43]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [44]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_students * 100
per_school_passing_reading = school_students_passing_reading / per_school_students * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_students * 100

In [58]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = {
    'School Type': school_type,
    'Total Students': per_school_students,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Math Score': per_school_math_avg,
    'Average Reading Score': per_school_reading_avg,
    '% Passing Math': per_school_passing_math,
    '% Passing Reading': per_school_passing_reading,
    '% Overall Passing': overall_passing_rate
}

per_school_summary_df = pd.DataFrame(per_school_summary)

# Formatting
#per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
#per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,[3124928],[628.0],77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,[1081356],[582.0],83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,[1884411],[639.0],76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,[1763916],[644.0],77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,[917500],[625.0],83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,[3022020],[652.0],77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,[248087],[581.0],83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,[1910635],[655.0],76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,[3094650],[650.0],77.072464,80.966394,66.057551,81.222432,53.539172
